In [ ]:
! pip list

In [ ]:
import torch
import torchvision

print(f"PyTorch version: {torch.__version__}")
print(f"torchvision version: {torchvision.__version__}")
print(f"Is CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
else:
    print("CUDA is not available.")

# 1. YOLO 객체 탐지 (미구현)

### yolov5 저장소 클론 (혹은 pip install ultralytics 로 yolov5 / yolov8 사용 가능)
git clone https://github.com/ultralytics/yolov5.git
cd yolov5
pip install -r requirements.txt


In [ ]:
import torch
from PIL import Image
import cv2

# 1) YOLO 모델 불러오기 (yolov5s 등 가중치)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# 2) 이미지 로드
img_path = 'test_image.jpg'
results = model(img_path, size=640)

# 3) 객체 중에서 'person' 혹은 'face' 클래스 필터링
#   -> yolov5s는 기본 COCO 클래스(80종) 중 'person'만 존재. 
#      'face'만 검출하려면 얼굴 전용 가중치나 custom model이 필요.
df = results.pandas().xyxy[0]  # detection 결과를 pandas로
for idx, row in df.iterrows():
    cls_name = row['name']   # 클래스 이름
    conf = row['confidence']
    if cls_name == 'person': 
        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
        # crop
        # ...

# 만약 얼굴만 검출하고 싶다면
# -> yolov5 face 전용 custom 모델 사용


In [ ]:
import cv2
import torchvision.transforms as T

transform = T.Compose([
    T.Resize((224, 224)),      # 모델 입력 크기에 맞게
    T.ToTensor(),
    T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])  # ImageNet 통계값
])

img_cv2 = cv2.imread(img_path)
crop_img = img_cv2[y1:y2, x1:x2, :]  # bounding box 영역 자르기
crop_img_rgb = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)

# PIL 객체로 변환 후 torch.Tensor로
crop_pil = Image.fromarray(crop_img_rgb)
input_tensor = transform(crop_pil)
input_tensor = input_tensor.unsqueeze(0)  # 배치 차원 추가 => shape (1, C, H, W)


# 2. 이진 분류 모델

### 1-데이터셋 및 DataLoader 구성

In [1]:
import torch
import torchvision.models as models
import torch.nn as nn

# -------------------------------------
# 0) device 설정
# -------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------------------------
# 1) 모델 선언 및 마지막 레이어 교체
# -------------------------------------
model = models.efficientnet_b0(pretrained=True)  # ImageNet 사전학습
num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, 2)  # Fake vs Real (2클래스)

# -------------------------------------
# 2) 데이터 로더 설정
# -------------------------------------
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import os

base_dir = os.getcwd()  # 현재 작업 디렉토리
train_dir = os.path.join(base_dir, 'Train')
val_dir   = os.path.join(base_dir, 'Validation')
test_dir  = os.path.join(base_dir, 'Test')

batch = 16
img_resized = (128, 128)

train_transform = transforms.Compose([
    transforms.Resize(img_resized),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(img_resized),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
val_dataset   = datasets.ImageFolder(root=val_dir, transform=test_transform)
test_dataset  = datasets.ImageFolder(root=test_dir, transform=test_transform)

train_loader  = DataLoader(train_dataset, batch_size=batch, shuffle=True,  num_workers=0)
val_loader    = DataLoader(val_dataset,   batch_size=batch, shuffle=False, num_workers=0)
test_loader   = DataLoader(test_dataset,  batch_size=batch, shuffle=False, num_workers=0)

# -------------------------------------
# 3) 학습 설정 (Loss, Optimizer 등)
# -------------------------------------
import torch.optim as optim

model = model.to(device)  # 여기서 device로 모델을 올림
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 10


c:\Users\user\.conda\envs\torch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\.conda\envs\torch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### 2-모델 훈련(파인튜닝) 및 검증

In [ ]:
# -------------------------------------
# 4) Training Loop
# -------------------------------------
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels).item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc  = correct / total

    # validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_images, val_labels in val_loader:
            val_images, val_labels = val_images.to(device), val_labels.to(device)
            val_outputs = model(val_images)
            v_loss = criterion(val_outputs, val_labels)
            val_loss += v_loss.item() * val_images.size(0)
            _, v_preds = torch.max(val_outputs, 1)
            val_correct += torch.sum(v_preds == val_labels).item()
            val_total += val_labels.size(0)

    val_epoch_loss = val_loss / val_total
    val_epoch_acc  = val_correct / val_total

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {epoch_loss:.4f} Train Acc: {epoch_acc:.4f} | "
          f"Val Loss: {val_epoch_loss:.4f} Val Acc: {val_epoch_acc:.4f}")

# -------------------------------------
# 5) Model Save
# -------------------------------------
pth_name = "binary_test_244.pth"
torch.save(model.state_dict(), pth_name)


Epoch [1/10] Train Loss: 0.1916 Train Acc: 0.9274 | Val Loss: 0.0893 Val Acc: 0.9683
Epoch [2/10] Train Loss: 0.0702 Train Acc: 0.9760 | Val Loss: 0.0935 Val Acc: 0.9657
Epoch [3/10] Train Loss: 0.0372 Train Acc: 0.9873 | Val Loss: 0.1516 Val Acc: 0.9532
Epoch [4/10] Train Loss: 0.0274 Train Acc: 0.9912 | Val Loss: 0.0958 Val Acc: 0.9706
Epoch [5/10] Train Loss: 0.0221 Train Acc: 0.9933 | Val Loss: 0.1597 Val Acc: 0.9600
Epoch [6/10] Train Loss: 0.0274 Train Acc: 0.9900 | Val Loss: 0.0730 Val Acc: 0.9742
Epoch [7/10] Train Loss: 0.0145 Train Acc: 0.9952 | Val Loss: 0.0635 Val Acc: 0.9801
Epoch [8/10] Train Loss: 0.0159 Train Acc: 0.9943 | Val Loss: 0.1042 Val Acc: 0.9689
Epoch [9/10] Train Loss: 0.0236 Train Acc: 0.9922 | Val Loss: 0.0587 Val Acc: 0.9833
Epoch [10/10] Train Loss: 0.0120 Train Acc: 0.9957 | Val Loss: 0.0546 Val Acc: 0.9840


### 3-테스트(추가 평가)

In [3]:
# -------------------------------------
# 6) Test
# -------------------------------------
model.eval()

test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * images.size(0)

        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels).item()
        total += labels.size(0)

avg_loss = test_loss / total
accuracy = correct / total

print(f"[Test] Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")


[Test] Loss: 0.6456, Accuracy: 0.8414


### 99-TorchScript(.pt) 병합 (모델 구조 + 가중치.pth)

In [ ]:
import torch
import torchvision.models as models
import torch.nn as nn

# 1) 모델 정의 (학습 때와 동일 구조)
model = models.efficientnet_b0(pretrained=False)
num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, 2)
model.load_state_dict(torch.load(pth_name, map_location="cpu"))
model.eval()

# 2) TorchScript 변환
model_name = "efficientnet_b0_deepfake_128.pt"
scripted_model = torch.jit.script(model)  # 또는 torch.jit.trace(model, example_input)
scripted_model.save(model_name)


c:\Users\user\.conda\envs\torch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\user\AppData\Local\Temp\ipykernel_19660\240586017.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.seri

## 외부 이미지 테스트

In [ ]:
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

# 1) device 설정 (GPU 사용 가능 시 GPU 할당)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2) 모델 구조 정의 (학습 때와 동일하게)
model = torch.jit.load("efficientnet_b0_deepfake_128.pt", map_location=device)
model.eval()

# 4) 이미지 전처리 transform (학습 때와 동일하게)
img_resized = (128, 128)

transform = transforms.Compose([
    transforms.Resize(img_resized),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# 5) 예측할 이미지 폴더 경로
base_dir = os.getcwd()  # 현재 작업 디렉토리
image_folder = os.path.join(base_dir, 'sample')

# 6) 폴더 내 이미지들에 대해 일괄 추론
with torch.no_grad():
    for filename in os.listdir(image_folder):
        # 파일 확장자 확인 (jpg, png 등)
        if filename.lower().endswith((".jpg", ".jpeg", ".png", ".bmp")):
            img_path = os.path.join(image_folder, filename)

            # 이미지 열기
            img = Image.open(img_path).convert('RGB')
            # 전처리
            img_tensor = transform(img).unsqueeze(0).to(device)  # shape: (1, C, H, W)

            # 모델 추론
            outputs = model(img_tensor)
            # 예: CrossEntropyLoss 기준 => argmax
            _, pred = torch.max(outputs, 1)

            # 0 -> Fake, 1 -> Real (ImageFolder 학습 기준)
            label_idx = pred.item()
            label_str = "Fake" if label_idx == 0 else "Real"

            print(f"[{filename}] => {label_str}")


[0000.png] => Fake
[0150.png] => Real
[0275.png] => Fake
[10_20241130135752_A_close-up_shot_of_a_teenager_non-binary_person_of.jpg] => Fake
[14_20241130135832_A_close-up_shot_of_a_elderly_genderqueer_person_of.jpg] => Fake
[god01.jpg] => Real
[god02.jpg] => Real
[god03.jpg] => Real


# 3. 다중 분류 모델

### 1-데이터셋 및 DataLoader 구성

In [1]:
import torch
import torchvision.models as models
import torch.nn as nn
import time

# -------------------------------------
# 0) device 설정
# -------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------------------------
# 1) 모델 선언 및 마지막 레이어 교체
# -------------------------------------
model = models.efficientnet_b0(pretrained=True)  # ImageNet 사전학습
num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, 3)  # 3-class (AIGenerated, Fake, Real)

# -------------------------------------
# 2) 데이터 로더 설정
# -------------------------------------
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import os

base_dir = os.getcwd()  # 현재 작업 디렉토리
train_dir = os.path.join(base_dir, 'Train')
val_dir   = os.path.join(base_dir, 'Validation')
test_dir  = os.path.join(base_dir, 'Test')

batch = 16
img_resized = (128, 128)

train_transform = transforms.Compose([
    transforms.Resize(img_resized),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(img_resized),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
val_dataset   = datasets.ImageFolder(root=val_dir, transform=test_transform)
test_dataset  = datasets.ImageFolder(root=test_dir, transform=test_transform)

train_loader  = DataLoader(train_dataset, batch_size=batch, shuffle=True,  num_workers=0, pin_memory=True)
val_loader    = DataLoader(val_dataset,   batch_size=batch, shuffle=False, num_workers=0, pin_memory=True)
test_loader   = DataLoader(test_dataset,  batch_size=batch, shuffle=False, num_workers=0, pin_memory=True)

# 클래스 인덱스 확인
print("Train Dataset class to index mapping:")
print(train_dataset.class_to_idx)


# -------------------------------------
# 3) 학습 설정 (Loss, Optimizer 등)
# -------------------------------------
import torch.optim as optim

model = model.to(device)  # 여기서 device로 모델을 올림
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 10


c:\Users\user\.conda\envs\torch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\.conda\envs\torch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Train Dataset class to index mapping:
{'AIGenerated': 0, 'Fake': 1, 'Real': 2}


### 2-모델 훈련(파인튜닝) 및 검증

In [2]:
from tqdm import tqdm

# -------------------------------------
# 4) Training Loop
# -------------------------------------
total_start_time = time.time()

for epoch in range(num_epochs):
    start_time = time.time()  # 에포크 시작 시간 기록
    
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels).item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc  = correct / total

    # validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_images, val_labels in val_loader:
            val_images, val_labels = val_images.to(device), val_labels.to(device)
            val_outputs = model(val_images)
            v_loss = criterion(val_outputs, val_labels)
            val_loss += v_loss.item() * val_images.size(0)
            _, v_preds = torch.max(val_outputs, 1)
            val_correct += torch.sum(v_preds == val_labels).item()
            val_total += val_labels.size(0)

    val_epoch_loss = val_loss / val_total
    val_epoch_acc  = val_correct / val_total
    
    end_time = time.time()  # 에포크 종료 시간 기록
    epoch_duration = end_time - start_time  # 에포크 소요 시간 계산

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {epoch_loss:.4f} Train Acc: {epoch_acc:.4f} | "
          f"Val Loss: {val_epoch_loss:.4f} Val Acc: {val_epoch_acc:.4f} | "
          f"Time: {epoch_duration:.2f} sec")

total_end_time = time.time()
total_duration = total_end_time - total_start_time
print(f"Total Training Time: {total_duration/60:.2f} minutes")

# -------------------------------------
# 5) Model Save
# -------------------------------------
pth_name = "binary_test_3class_128.pth"
torch.save(model.state_dict(), pth_name)


Epoch 1/10 - Training: 100%|██████████| 551/551 [01:12<00:00,  7.56it/s]


Epoch [1/10] Train Loss: 0.2722 Train Acc: 0.9033 | Val Loss: 0.1306 Val Acc: 0.9569 | Time: 97.84 sec


Epoch 2/10 - Training: 100%|██████████| 551/551 [00:40<00:00, 13.58it/s]


Epoch [2/10] Train Loss: 0.0966 Train Acc: 0.9651 | Val Loss: 0.1105 Val Acc: 0.9592 | Time: 58.42 sec


Epoch 3/10 - Training: 100%|██████████| 551/551 [00:40<00:00, 13.61it/s]


...

Epoch [8/10] Train Loss: 0.0220 Train Acc: 0.9931 | Val Loss: 0.1009 Val Acc: 0.9649 | Time: 58.78 sec


Epoch 9/10 - Training: 100%|██████████| 551/551 [00:41<00:00, 13.27it/s]


Epoch [9/10] Train Loss: 0.0136 Train Acc: 0.9958 | Val Loss: 0.1031 Val Acc: 0.9681 | Time: 59.57 sec


Epoch 10/10 - Training: 100%|██████████| 551/551 [00:40<00:00, 13.55it/s]


Epoch [10/10] Train Loss: 0.0126 Train Acc: 0.9957 | Val Loss: 0.0932 Val Acc: 0.9706 | Time: 58.40 sec
Total Training Time: 10.44 minutes


### 3-테스트(추가 평가)

In [3]:
# -------------------------------------
# 6) Test
# -------------------------------------
model.eval()

test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * images.size(0)

        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels).item()
        total += labels.size(0)

avg_loss = test_loss / total
accuracy = correct / total

print(f"[Test] Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")


[Test] Loss: 0.5539, Accuracy: 0.8655


### 99-TorchScript(.ptl) 병합 (모델 구조 + 가중치.pth)

#### ai binary classification

In [2]:
import torch
import torchvision.models as models
import torch.nn as nn
from torch.utils.mobile_optimizer import optimize_for_mobile

# 모델 로드 및 TorchScript 변환
model = models.efficientnet_b0(pretrained=False)
num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, 2)

model.load_state_dict(torch.load("ai_gen_b128_e10.pth", map_location="cpu"))
model.eval()

# 1) 스크립팅
scripted_model = torch.jit.script(model)

# 2) 모바일 최적화
optimized_model = optimize_for_mobile(scripted_model)

# 3) Lite Interpreter용으로 저장
optimized_model._save_for_lite_interpreter("efficientnet_b0_ai_gen.ptl")


c:\Users\user\.conda\envs\torch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\.conda\envs\torch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\user\AppData\Local\Temp\ipykernel_20412\3526195180.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` 

In [1]:
import torch
import torchvision.models as models
import torch.nn as nn
from torch.utils.mobile_optimizer import optimize_for_mobile

# 1) 모델 정의 (학습 때와 동일 구조)
model = models.efficientnet_b0(pretrained=False)
num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, 3)
model.load_state_dict(torch.load(pth_name, map_location="cpu"))
model.eval()

# 2) TorchScript 변환
model_name = "efficientnet_b0_3class_128.pt"
scripted_model = torch.jit.script(model)  # 또는 torch.jit.trace(model, example_input)
scripted_model.save(model_name)

# 모델 로드 및 TorchScript 변환
model = models.efficientnet_b0(pretrained=False)
num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, 3)

model.load_state_dict(torch.load("face_3class_s128_e10.pth", map_location="cpu"))
model.eval()

# 1) 스크립팅
scripted_model = torch.jit.script(model)

# 2) 모바일 최적화
optimized_model = optimize_for_mobile(scripted_model)

# 3) Lite Interpreter용으로 저장
optimized_model._save_for_lite_interpreter("efficientnet_b0_deepfake_s128.ptl")


c:\Users\user\.conda\envs\torch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\.conda\envs\torch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\user\AppData\Local\Temp\ipykernel_21112\3887797595.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` 

## 외부 이미지 테스트

In [1]:
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

# 1) device 설정 (GPU 사용 가능 시 GPU 할당)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2) 모델 구조 정의 (학습 때와 동일하게)
model = torch.jit.load("efficientnet_b0_3class_128.pt", map_location=device)
model.eval()

# 4) 이미지 전처리 transform (학습 때와 동일하게)
img_resized = (128, 128)

transform = transforms.Compose([
    transforms.Resize(img_resized),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# 5) 예측할 이미지 폴더 경로
base_dir = os.getcwd()  # 현재 작업 디렉토리
image_folder = os.path.join(base_dir, 'sample')

# 6) 폴더 내 이미지들에 대해 일괄 추론
with torch.no_grad():
    for filename in os.listdir(image_folder):
        # 파일 확장자 확인 (jpg, png 등)
        if filename.lower().endswith((".jpg", ".jpeg", ".png", ".bmp")):
            img_path = os.path.join(image_folder, filename)

            # 이미지 열기
            img = Image.open(img_path).convert('RGB')
            # 전처리
            img_tensor = transform(img).unsqueeze(0).to(device)  # shape: (1, C, H, W)

            # 모델 추론
            outputs = model(img_tensor)
            # 예: CrossEntropyLoss 기준 => argmax
            _, pred = torch.max(outputs, 1)

            # 0 -> Fake, 1 -> Real (ImageFolder 학습 기준)
            label_idx = pred.item()
            label_str = "AIGenerated" if label_idx == 0 else ("Fake" if label_idx == 1 else "Real")

            print(f"[{filename}] => {label_str}")


[64c47081-446f-4ff7-b811-3bb55cabcb35.jpg] => Fake
[god01.jpg] => Real
[god02.jpg] => Real
[pexels-photo-2379005.jpeg] => Real


# 3. 통합 (yolo + detection) (미구현)

In [ ]:
import torch
import cv2
from PIL import Image
import torchvision.transforms as T

# 1) YOLO 로드
face_detector = torch.hub.load('ultralytics/yolov5', 'custom', 'path/to/yolov5_face.pt')  
# 혹은 yolov5s, yolov8 등

# 2) 딥페이크 분류 모델 로드
model = models.efficientnet_b0()
# 분류기 교체
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
model.load_state_dict(torch.load('deepfake_classifier.pth'))
model.eval().cuda()  # GPU 사용

transform = T.Compose([
    T.Resize((224,224)),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])

def detect_and_classify(img_path):
    # 1) 얼굴 검출
    results = face_detector(img_path, size=640)
    df = results.pandas().xyxy[0]

    img_cv2 = cv2.imread(img_path)
    h, w, _ = img_cv2.shape

    for idx, row in df.iterrows():
        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
        cls_conf = row['confidence']

        # 2) 얼굴 영역 crop
        crop_img = img_cv2[y1:y2, x1:x2, :]
        crop_img_rgb = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(crop_img_rgb)
        
        input_tensor = transform(pil_img).unsqueeze(0).cuda()

        # 3) 딥페이크 분류
        with torch.no_grad():
            outputs = model(input_tensor)
            probs = torch.softmax(outputs, dim=1)  # CrossEntropyLoss 기반
            # probs[0,0] = fake 확률, probs[0,1] = real 확률 (ImageFolder 순서에 따라 다름)
            # 만약 fake 폴더가 index=0, real 폴더가 index=1이라면 아래와 같이
            fake_prob = probs[0,0].item()
            real_prob = probs[0,1].item()

        print(f"Face @({x1}, {y1}, {x2}, {y2}) => Fake: {fake_prob:.3f}, Real: {real_prob:.3f}")

# 테스트
detect_and_classify('test.jpg')


In [14]:
import os
import shutil

# 원본 경로와 복사할 경로 지정
source_root = r"C:\Users\user\Desktop\cropped_images"
destination_root = r"C:\Users\user\Desktop\selected_images"

# 대상 경로가 없으면 생성
os.makedirs(destination_root, exist_ok=True)

# 이미지 순차 이름 설정
counter = 1

# 각 폴더 내부 이미지를 5개씩 복사
for folder_name in os.listdir(source_root):
    folder_path = os.path.join(source_root, folder_name)

    # 폴더인지 확인
    if os.path.isdir(folder_path):
        images = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
        
        # 5개만 선택하여 복사
        for i, image in enumerate(images[-4:-1]):
            source_image_path = os.path.join(folder_path, image)

            # 순차 이름 생성
            destination_image_name = f"crop_{counter:04d}{os.path.splitext(image)[1]}"
            destination_image_path = os.path.join(destination_root, destination_image_name)

            # 복사 수행
            shutil.copy2(source_image_path, destination_image_path)

            print(f"{source_image_path} -> {destination_image_path}")

            # 카운터 증가
            counter += 1

print("이미지 복사가 완료되었습니다.")


C:\Users\user\Desktop\cropped_images\000_003\0300.png -> C:\Users\user\Desktop\selected_images\crop_0001.png
C:\Users\user\Desktop\cropped_images\000_003\0325.png -> C:\Users\user\Desktop\selected_images\crop_0002.png
C:\Users\user\Desktop\cropped_images\000_003\0350.png -> C:\Users\user\Desktop\selected_images\crop_0003.png
C:\Users\user\Desktop\cropped_images\001_870\0375.png -> C:\Users\user\Desktop\selected_images\crop_0004.png
C:\Users\user\Desktop\cropped_images\001_870\0400.png -> C:\Users\user\Desktop\selected_images\crop_0005.png
C:\Users\user\Desktop\cropped_images\001_870\0425.png -> C:\Users\user\Desktop\selected_images\crop_0006.png
C:\Users\user\Desktop\cropped_images\002_006\0600.png -> C:\Users\user\Desktop\selected_images\crop_0007.png
C:\Users\user\Desktop\cropped_images\002_006\0625.png -> C:\Users\user\Desktop\selected_images\crop_0008.png
C:\Users\user\Desktop\cropped_images\002_006\0650.png -> C:\Users\user\Desktop\selected_images\crop_0009.png
C:\Users\user\Deskt